# Análise PySpark — Seasons e Constructors (F1)

Este notebook carrega os CSVs `seasons.csv` e `constructors.csv` de `bronze/dados_originais`, executa uma análise exploratória (duplicados, nulos, distintos, consistência) e gera gráficos (barras, scatter) e regressão linear (linha reta) usando PySpark.

In [2]:
import os, sys, subprocess
from pyspark.sql import SparkSession

# garante que Spark vai usar o mesmo Python do driver
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# se já tiver Spark rodando, para
prev = SparkSession.getActiveSession()
if prev is not None:
    prev.stop()

# cria sessão Spark
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("Bronze-Analytics")
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow")
    .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark OK =>", spark.version)


25/09/24 17:41:09 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.ja

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:334)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a

In [1]:
# Inicialização: Spark + utilitários de path
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

import matplotlib.pyplot as plt
from pathlib import Path

spark = (
    SparkSession.builder
    .appName("F1-EDA-PySpark")
    .getOrCreate()
)

def find_base_dir():
    cwd = Path.cwd()
    for p in [cwd, *cwd.parents]:
        candidate = p / 'bronze' / 'dados_originais'
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Não encontrei 'bronze/dados_originais' a partir do diretório atual.")

BASE = find_base_dir()
SEASONS_PATH = str(BASE / 'seasons.csv')
CONSTRUCTORS_PATH = str(BASE / 'constructors.csv')
SEASONS_PATH, CONSTRUCTORS_PATH


25/09/24 17:40:48 WARN Utils: Your hostname, MacBook-Air-de-Kaleb.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
25/09/24 17:40:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/24 17:40:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:334)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a

In [ ]:
# Leitura dos CSVs com schema inferido
seasons_df = (
    spark.read.option('header', True).option('inferSchema', True).csv(SEASONS_PATH)
)
constructors_df = (
    spark.read.option('header', True).option('inferSchema', True).csv(CONSTRUCTORS_PATH)
)

seasons_df.printSchema()
constructors_df.printSchema()

print('Seasons linhas:', seasons_df.count())
print('Constructors linhas:', constructors_df.count())


## Funções auxiliares de EDA e Gráficos

In [ ]:
def contar_duplicados_linhas(df):
    total = df.count()
    unicos = df.dropDuplicates().count()
    return total - unicos

def duplicados_por_chave(df, chaves, n=20):
    return (
        df.groupBy(*chaves).count().where(F.col('count') > 1).orderBy(F.col('count').desc()).limit(n)
    )

def contagem_nulos_vazios(df):
    exprs = []
    for c in df.columns:
        exprs.append(F.sum(F.col(c).isNull().cast('int')).alias(f'{c}__nulls'))
        # Considera string vazia como "vazio"
        exprs.append(
            F.sum((F.col(c).cast('string') == '').cast('int')).alias(f'{c}__empties')
        )
    return df.agg(*exprs)

def contagem_distintos(df):
    exprs = [F.countDistinct(c).alias(c) for c in df.columns]
    return df.agg(*exprs)

def add_row_index(df, order_cols=None, idx_name='idx'):
    if order_cols is None:
        order_cols = df.columns
    win = Window.orderBy([F.col(c).asc_nulls_last() for c in order_cols])
    return df.withColumn(idx_name, F.row_number().over(win).cast('double'))

def plot_bar_from_spark(df, x_col, y_col, title, xlabel=None, ylabel=None, rot=45):
    pdf = df.select(x_col, y_col).toPandas()
    plt.figure(figsize=(10, 5))
    plt.bar(pdf[x_col], pdf[y_col], color='#4C78A8')
    plt.title(title)
    plt.xlabel(xlabel or x_col)
    plt.ylabel(ylabel or y_col)
    plt.xticks(rotation=rot)
    plt.tight_layout()
    plt.show()

def scatter_com_regressao(df, y_col, order_cols=None, title=None):
    # X = índice de linha (1..N) com ordenação definida
    df_idx = add_row_index(df.select(y_col), order_cols=order_cols, idx_name='x_idx')
    df_idx = df_idx.withColumn('label', F.col(y_col).cast('double'))

    assembler = VectorAssembler(inputCols=['x_idx'], outputCol='features')
    feat_df = assembler.transform(df_idx).select('x_idx', 'label', 'features')

    lr = LinearRegression(featuresCol='features', labelCol='label')
    model = lr.fit(feat_df)
    pred = model.transform(feat_df).select('x_idx', 'label', 'prediction')

    r2 = getattr(model.summary, 'r2', None)
    slope = float(model.coefficients[0])
    intercept = float(model.intercept)

    pdf = pred.orderBy('x_idx').toPandas()
    plt.figure(figsize=(8, 5))
    plt.scatter(pdf['x_idx'], pdf['label'], s=12, alpha=0.6, color='#72B7B2', label='Dados')
    plt.plot(pdf['x_idx'], pdf['prediction'], color='#F58518', linewidth=2, label='Regressão')
    ttl = title or f'Regressão linear: Y={y_col} vs X=índice'
    if r2 is not None:
        ttl += f' (slope={slope:.3f}, R²={r2:.3f})'
    else:
        ttl += f' (slope={slope:.3f})'
    plt.title(ttl)
    plt.xlabel('X = índice (1..N)')
    plt.ylabel(f'Y = {y_col}')
    plt.legend()
    plt.tight_layout()
    plt.show()

    return model


## Seasons — EDA

In [ ]:
seasons_df.show(5, truncate=False)
print('Schema Seasons:')
seasons_df.printSchema()

print('Total de linhas (Seasons):', seasons_df.count())
print('Duplicados de linha (Seasons):', contar_duplicados_linhas(seasons_df))

print('Duplicados por chave: year')
duplicados_por_chave(seasons_df, ['year']).show(truncate=False)

print('Contagem de nulos/vazios por coluna (Seasons):')
contagem_nulos_vazios(seasons_df).show(truncate=False)

print('Contagem de distintos por coluna (Seasons):')
contagem_distintos(seasons_df).show(truncate=False)

# Validação simples de intervalo de ano (ex.: >= 1950)
seasons_df.select(
    F.min('year').alias('min_year'), F.max('year').alias('max_year')
).show()


### Seasons — Gráficos

In [ ]:
# Barras: número de temporadas por década
seasons_decade = (
    seasons_df.withColumn('decade', (F.col('year')/10).cast('int')*10)
        .groupBy('decade').count().orderBy('decade')
)
plot_bar_from_spark(seasons_decade, 'decade', 'count', 'Temporadas por década', 'Década', 'Qtd')

# Scatter + regressão: X = índice (1..N), Y = year
_ = scatter_com_regressao(seasons_df, y_col='year', order_cols=['year'], title='Seasons: regressão (índice vs year)')


## Constructors — EDA

In [ ]:
constructors_df.show(5, truncate=False)
print('Schema Constructors:')
constructors_df.printSchema()

print('Total de linhas (Constructors):', constructors_df.count())
print('Duplicados de linha (Constructors):', contar_duplicados_linhas(constructors_df))

print('Duplicados por chave: constructorId')
duplicados_por_chave(constructors_df, ['constructorId']).show(truncate=False)
print('Duplicados por chave: constructorRef')
duplicados_por_chave(constructors_df, ['constructorRef']).show(truncate=False)

print('Contagem de nulos/vazios por coluna (Constructors):')
contagem_nulos_vazios(constructors_df).show(truncate=False)

print('Contagem de distintos por coluna (Constructors):')
contagem_distintos(constructors_df).show(truncate=False)

# Checagem simples de URLs não vazias / formato básico
constructors_df.select(
    F.count(F.when(F.col('url').isNull() | (F.col('url') == ''), 1)).alias('urls_vazias'),
    F.count(F.when(~F.col('url').rlike('^https?://'), 1)).alias('urls_nao_http')
).show()


### Constructors — Gráficos

In [ ]:
# Barras: Top nacionalidades por número de construtores
nat_counts = (
    constructors_df.groupBy('nationality').count().orderBy(F.col('count').desc(), F.col('nationality'))
)
top_nat = nat_counts.limit(15).orderBy(F.col('count').desc())
plot_bar_from_spark(top_nat, 'nationality', 'count', 'Top 15 nacionalidades (constructors)', 'Nacionalidade', 'Qtd', rot=60)

# Scatter + regressão: X = índice (1..N), Y = constructorId
_ = scatter_com_regressao(constructors_df, y_col='constructorId', order_cols=['constructorId'], title='Constructors: regressão (índice vs constructorId)')


## Ações de limpeza sugeridas (exemplos em PySpark)

In [ ]:
# Remoção de duplicados por chave (Seasons): manter primeiro por year
seasons_dedup = seasons_df.dropDuplicates(['year'])

# Remoção de duplicados por chave (Constructors): manter primeiro por constructorId e constructorRef
constructors_dedup = constructors_df.dropDuplicates(['constructorId', 'constructorRef'])

# Normalização simples de strings (trim)
constructors_clean = (
    constructors_dedup
    .withColumn('name', F.trim(F.col('name')))
    .withColumn('nationality', F.trim(F.col('nationality')))
)

# Preenchimento opcional de URLs vazias com None (nulo)
constructors_clean = constructors_clean.withColumn(
    'url', F.when((F.col('url') == '') | F.col('url').isNull(), F.lit(None)).otherwise(F.col('url'))
)

seasons_dedup.show(5)
constructors_clean.show(5)
